# Main changes since DemoV1:
---

- `optimize` flag, for actual optional optimizations
    - Make the deletion of gates with no amplitude (fill delete_gates_with_no_amplitude(...))
    - Make bunching of consecutive Drag Gates if phis equal (fill bunch_drag_gates(... only_same_phi=True))
- Make Transpilation/execute/compile only for single circuits (not list(circuits))
- Pass a all the transpilation kwargs as a dataclass `DigitalTranspilationConfig` in `execute`'s
-  Make more clear what are keys/values in the dict of the reported/docstring layouts
- State clearly the final ordering of qubits (@Ameer)
    - log both init_layout (placement) & final_layout (placement+routing)
 

# Imports:
---

In [1]:
# Qililab imports # noqa: CPY001
import qililab as ql  # noqa: I001
from qililab.digital.circuit_transpiler import CircuitTranspiler

# Qibo imports
from qibo import gates, Circuit

# Create platform:
runcard = "/home/guille.abad/Github/qililab/tests/calibration/galadriel.yml"
platform = ql.build_platform(runcard)

# Create transpiler:
transpiler = CircuitTranspiler(platform.digital_compilation_settings)

# Create circuit:
# c = Circui.Xt(4)
# c.add(gates.CNOT(1, 0))
# c.add(gates.X(0))
# c.add(gates.X(3))
# c.add(gates.CNOT(3, 2))
# c.add(gates.X(0))
# c.add(gates(3))

2025-02-06 15:40:05,401 - qm - INFO     - Starting session: 6bea912c-46dc-4507-bf0d-38eb42908e60


In [2]:
c2 = Circuit(3)
c2.add(gates.X(0))
c2.add(gates.CNOT(0,2))
c2.add(gates.X(0))
c2.add(gates.X(2))
c2.add(gates.CNOT(1,2))
c2.add(gates.X(1))
c2.add(gates.CNOT(0,1))
c2.add(gates.X(1))

# for i in range(2):
#     c2.add(gates.M(i))

In [3]:
platform.digital_compilation_settings.topology

[(0, 1), (1, 2)]

In [4]:
def print_info(routed_circuit: list[gates.Gate], qubits: int):
    print("\nManual Printing of info:")
    print("Final Routed circuit:", [(gate.name, gate.qubits) for gate in routed_circuit])
    print("Original circuit had 4 qubits, the new one will have as much as the platform has: ", qubits)

# Automatic transpilation with `ql.execute()`:
---

In [8]:
from qililab.digital import DigitalTranspilationConfig

# Create transpilation config:
transpilation = DigitalTranspilationConfig(routing=True, optimize=False)

# Execute with automatic transpilation:
probabilities = ql.execute(c2, runcard=runcard, transpilation_config=transpilation)

[qililab] [0.28.0|INFO|2025-02-06 15:46:47]: Connecting to instrument cluster_controller_0.
[qililab] [0.28.0|INFO|2025-02-06 15:46:50]: Connected to the instruments
[qililab] [0.28.0|INFO|2025-02-06 15:46:50]: Initial setup to instrument QRM-RF1.
[qililab] [0.28.0|INFO|2025-02-06 15:46:50]: Initial setup to instrument QRM1.
[qililab] [0.28.0|INFO|2025-02-06 15:46:51]: Initial setup to instrument QCM1.
[qililab] [0.28.0|INFO|2025-02-06 15:46:51]: Initial setup to instrument QCM2.
[qililab] [0.28.0|INFO|2025-02-06 15:46:52]: Initial setup to instrument QCM-RF1.
[qililab] [0.28.0|INFO|2025-02-06 15:46:52]: Initial setup applied to the instruments
[qililab] [0.28.0|INFO|2025-02-06 15:46:52]: Turn on instrument QRM-RF1.
[qililab] [0.28.0|INFO|2025-02-06 15:46:52]: Turn on instrument QRM1.
[qililab] [0.28.0|INFO|2025-02-06 15:46:52]: Turn on instrument QCM1.
[qililab] [0.28.0|INFO|2025-02-06 15:46:52]: Turn on instrument QCM2.
[qililab] [0.28.0|INFO|2025-02-06 15:46:52]: Turn on instrument 

  0%|          | 0/1 [00:00<?, ?it/s]

[Qibo 0.2.15|ERROR|2025-02-06 15:47:07]: Cannot access final state before the circuit is executed.
[Qibo 0.2.15|ERROR|2025-02-06 15:47:10]: Cannot access final state before the circuit is executed.
[qililab] [0.28.0|INFO|2025-02-06 15:47:41]: The best found routing, has 1 swaps.
[qililab] [0.28.0|INFO|2025-02-06 15:47:41]: [0, 2, 1]: Initial Re-mapping of the Original Logical Qubits (l_q), in the Physical Circuit: [l_q in wire 0, l_q in wire 1, ...].
[qililab] [0.28.0|INFO|2025-02-06 15:47:41]: [2, 0, 1]: Final Re-mapping (Initial + SWAPs routing) of the Original Logical Qubits (l_q), in the Physical Circuit: [l_q in wire 0, l_q in wire 1, ...].
[Qibo 0.2.15|ERROR|2025-02-06 15:47:41]: Cannot access final state before the circuit is executed.
[qililab] [0.28.0|INFO|2025-02-06 15:47:56]: The best found routing, has 1 swaps.
[qililab] [0.28.0|INFO|2025-02-06 15:47:56]: [0, 2, 1]: Initial Re-mapping of the Original Logical Qubits (l_q), in the Physical Circuit: [l_q in wire 0, l_q in wire

In [7]:
print(probabilities.acquisitions())

   acquisition_index  bins_index    i    q  amplitude     phase
0                  0           0  1.0  1.0     3.0103  0.785398
1                  1           0  0.0  0.0       -inf  0.000000
2                  2           0  0.0  0.0       -inf  0.000000


# Automatic transpilation with `platform.execute()`:
---

In [ ]:
from qibo.transpiler.placer import Random
from qibo.transpiler.router import Sabre, StarConnectivityRouter

# Create transpilation config:
transp_config = DigitalTranspilationConfig(routing=True, optimize=False, router=Sabre, placer=Random)
c.wire_names = [0, 1, 2, 4, 3]

# Execute with automatic transpilation:
result = platform.execute(c2, num_avg=1000, transpilation_config=transp_config)

# Manual Transpilation:
---

In [33]:
import pytest

# Fails since, CNOT(0,1) is not available, and routing=False!
with pytest.raises(KeyError):
    # Default Transpilation (with ReverseTraversal, Sabre, platform's connectivity and optimize = False):
    transpiled_circuit, final_layouts = transpiler.transpile_circuit(c2)


In [ ]:
# Default Transpilation (with ReverseTraversal, Sabre, platform's connectivity and optimize = True):
transpiled_circuit, final_layouts = transpiler.transpile_circuit(c2, transpilation_config=transp_config)

In [ ]:
# Or another case, not doing optimization for some reason, and with Non-Default placer:
transp_config.optimize = True
transpiled_circuit, final_layout = transpiler.transpile_circuit(c2, transpilation_config=transp_config)

# Manual Routing:
---

In [ ]:
# Create circuit to transpile:
c = Circuit(4)
c.add(gates.CNOT(1, 0))
c.add(gates.CNOT(2, 3))

# Default Transpilation (ReverseTraversal, Sabre , 10 iterations and platform connectivity):
routed_circuit, qubits, final_layouts = transpiler.route_circuit(c)
print_info(routed_circuit, qubits)

In [ ]:
# Non-Default Placer and Router specified, with non-instantiated classes:
routed_circuit, quubits, final_layouts = transpiler.route_circuit(c, placer=Random, router=StarConnectivityRouter)
print_info(routed_circuit, qubits)

In [ ]:
# Passing an instance for the placer and specifying Router with class and its kwargs:
placer = Random(samples=10, seed=1234)
routed_circuit, qubits, final_layouts = transpiler.route_circuit(c, placer=placer, router=(Sabre, {"lookahead": 2}))
print_info(routed_circuit, qubits)